In [ ]:
# thi file contains all preprocessing and mysql upload for the initial database state upload
# data is uploaded from mysql to bigquery

### Data Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns',500) 
plt.style.use('seaborn-v0_8-colorblind') #a style that can be used for plots
sns.set_style('whitegrid')

In [ ]:
df = pd.read_csv('data/heart_disease_uci.csv', index_col = 0, header = 0)
df.info()

In [ ]:
# drop dataset the data is from 
# and drop 'ca' - we are missing > 60% of this values and we dont need to fill it in
df = df.drop(['dataset', 'ca'], axis=1)

In [ ]:
# rename columns for my sanity
df.rename(columns = {'cp':'Chest_Pain_Type', 'trestbps':'Resting_Blood_Pressure','chol':'serum_cholesterol','fbs':'Fasting_Blood_Sugar','restecg':'Resting_Electrocardiographic','thalch':'Maximum_Heart_Rate','exang':'Exercise_Induced_Angina','oldpeak':'ST_Depression_Induced','thal':'Heart_Condition'}, inplace = True)

In [ ]:
# fill Resting_Blood_Pressure n/a with mean values
df["Resting_Blood_Pressure"].fillna(df["Resting_Blood_Pressure"].mean(), inplace=True)

# fill serum_cholesterol n/a with mean values
df["serum_cholesterol"].fillna(df["serum_cholesterol"].mean(), inplace=True)

# fill age n/a with mean values
df["age"].fillna(df["age"].mean(), inplace=True)

# fill Fasting_Blood_Sugar n/a with mean valuesForst. 
df["Fasting_Blood_Sugar"].fillna(df["Fasting_Blood_Sugar"].astype(float).mean(), inplace=True)

# fill Maximum_Heart_Rate n/a with mean values
df["Maximum_Heart_Rate"].fillna(df["Maximum_Heart_Rate"].mean(), inplace=True)

# fill ST_Depression_Induced n/a with mean values
df["ST_Depression_Induced"].fillna(df["ST_Depression_Induced"].mean(), inplace=True)

In [ ]:
# function to help fill in categorical data
def na_randomfill(series):
    na_mask = pd.isnull(series)   # boolean mask for null values
    n_null = na_mask.sum()        # number of nulls in the Series
    
    if n_null == 0:
        return series             # if there are no nulls, no need to resample
    
    # Randomly sample the non-null values from our series
    #  only sample this Series as many times as we have nulls 
    fill_values = series[~na_mask].sample(n=n_null, replace=True, random_state=0)

    # This ensures our new values will replace NaNs in the correct locations
    fill_values.index = series.index[na_mask]
    
    return series.fillna(fill_values) 

In [ ]:
# fill in the rest of the values with distributed random values
df["Resting_Electrocardiographic"] = na_randomfill(df["Resting_Electrocardiographic"])
df["Exercise_Induced_Angina"] = na_randomfill(df["Exercise_Induced_Angina"])
df["ST_Depression_Induced"] = na_randomfill(df["ST_Depression_Induced"])
df["slope"] = na_randomfill(df["slope"])
df["Heart_Condition"] = na_randomfill(df["Heart_Condition"])

### Generate Data/Connect and Load Data to MySQL

In [ ]:
#!pip install names

In [ ]:
#!pip install random_address

In [ ]:
import names
import random
import random_address
from datetime import date
import datetime

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  port=3306,
  password="PASSWORD",
  database="cardiology"
)

In [ ]:
mycursor = mydb.cursor()

In [ ]:
#random number generator
def rand_x_digit_num(x, leading_zeroes=True):
    """Return an X digit number, leading_zeroes returns a string, otherwise int"""
    if not leading_zeroes:
        # wrap with str() for uniform results
        return random.randint(10**(x-1), 10**x-1)  
    else:
        if x > 6000:
            return ''.join([str(random.randint(0, 9)) for i in xrange(x)])
        else:
            return '{0:0{x}d}'.format(random.randint(0, 10**x-1), x=x)      

In [ ]:
#area codes from phone number
stateAreaCode = {"AK": [907],
            "AL": [205, 251, 256, 334],
            "AZ": [479, 501, 870],
            "AR": [480, 520, 602, 623, 928],
            "CA": [209, 213, 310, 323, 408, 415, 510, 530, 559, 562, 619, 626, 650, 661, 707, 714, 760, 805, 818, 831, 858, 909, 916, 925, 949, 951],
            "CO": [303, 719, 970],
            "CT": [203, 860],
            "DC": [202],
            "DE": [302],
            "FL": [239, 305, 321, 352, 386, 407, 561, 727, 772, 813, 850, 863, 904, 941, 954],
            "GA": [229, 404, 478, 706, 770, 912],
            "HI": [808],
            "IA": [319, 515, 563, 641, 712],
            "ID": [208],
            "IL": [217, 309, 312, 618, 630, 708, 773, 815, 847],
            "IN": [219, 260, 317, 574, 765, 812],
            "KS": [316, 620, 785, 913],
            "KY": [270, 502, 606, 859],
            "LA": [225, 318, 337, 504, 985],
            "MA": [413, 508, 617, 781, 978],
            "MD": [301, 410],
            "ME": [207],
            "MI": [231, 248, 269, 313, 517, 586, 616, 734, 810, 906, 989],
            "MN": [218, 320, 507, 612, 651, 763, 952],
            "MO": [314, 417, 573, 636, 660, 816],
            "MS": [228, 601, 662],
            "MT": [406],
            "NC": [252, 336, 704, 828, 910, 919],
            "ND": [701],
            "NE": [308, 402],
            "NH": [603],
            "NJ": [201, 609, 732, 856, 908, 973],
            "NM": [505, 575],
            "NV": [702, 775],
            "NY": [212, 315, 516, 518, 585, 607, 631, 716, 718, 845, 914],
            "OH": [216, 330, 419, 440, 513, 614, 740, 937],
            "OK": [405, 580, 918],
            "OR": [503, 541],
            "PA": [215, 412, 570, 610, 717, 724, 814],
            "RI": [401],
            "SC": [803, 843, 864],
            "SD": [605],
            "TN": [423, 615, 731, 865, 901, 931],
            "TX": [210, 214, 254, 281, 325, 361, 409, 432, 512, 713, 806, 817, 830, 903, 915, 936, 940, 956, 972, 979],
            "UT": [435, 801],
            "VA": [276, 434, 540, 703, 757, 804],
            "VT": [802],
            "WA": [206, 253, 360, 425, 509],
            "WI": [262, 414, 608, 715, 920],
            "WV": [304],
            "WY": [307]
}

In [ ]:
patient_sql = "INSERT INTO PATIENT (SSN, Fname, Lname, gender, age, street, city, state, zip, phone, doctorID, emergencyNum, emergencyName) VALUES (%s, %s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s)"
medical_sql = "INSERT INTO MEDICAL_DATA (SSN, apptDate, cp, trestbps, chol, fbs, restecg, thalch, exang, oldpeak, slope, thal, evaluation) VALUES (%s, %s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s)"
doctor_sql = "INSERT INTO DOCTOR (Fname, Lname, phone, office) VALUES (%s, %s, %s, %s)"

In [ ]:
# insert doctor data
for i in range(20): 
    first = names.get_first_name()
    last = names.get_last_name()
    email = first + "." + last + "@cardiology.com"
    phone = rand_x_digit_num(7)
    office_num = 300+i
    doctor_val = (first, last, phone, office_num) 
    mycursor.execute(doctor_sql, doctor_val)
    mydb.commit()

In [ ]:
for index, row in df.iterrows():
    SSN = rand_x_digit_num(9)
    first = names.get_first_name(gender=row['sex'])
    last = names.get_last_name()
    flag = True
    address = random_address.real_random_address()
    while(flag):
        try:
            street = address['address1']
            city = address['city']
            state = address['state']
            zipCode = address['postalCode']
            flag = False
        except KeyError: 
            address = random_address.real_random_address()
            flag = True
            
    phone_number = int(str(stateAreaCode[address["state"]][random.randint(0,len(stateAreaCode[address["state"]])-1)])+rand_x_digit_num(7))
    emergency_number = int(str(stateAreaCode[address["state"]][random.randint(0,len(stateAreaCode[address["state"]])-1)])+rand_x_digit_num(7))
    emergency_name = names.get_full_name()
    doctorID = (index%20)+1
    patient_val = (SSN, first, last, row['sex'], row['age'], street, city, state, zipCode, phone_number, doctorID, emergency_number, emergency_name)
    mycursor.execute(patient_sql, patient_val)
    mydb.commit()
    
    medical_val = (SSN, date.today(), row['Chest_Pain_Type'], row['Resting_Blood_Pressure'], row['serum_cholesterol'], row['Fasting_Blood_Sugar'], row['Resting_Electrocardiographic'], 
                   row['Maximum_Heart_Rate'], row['Exercise_Induced_Angina'], row['ST_Depression_Induced'], row['slope'], row['Heart_Condition'], row['num'])
    
    mycursor.execute(medical_sql, medical_val)
    mydb.commit()
    

In [ ]:
#close connection
mydb.close()